<a href="https://colab.research.google.com/github/Akhila636636/EventEase/blob/main/Final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%writefile reqirements.txt
streamlit
pandas
numpy
xgboost
folium
streamlit-folium
requests
retry-requests
matplotlib
scikit-learn

Overwriting reqirements.txt


In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import folium
from streamlit_folium import folium_static
from xgboost import XGBRegressor
import requests
from retry_requests import retry
from datetime import datetime, timedelta
import random
import matplotlib.pyplot as plt
import json
import os
from folium.plugins import AntPath

# --- CRITICAL ENVIRONMENT FIX (For Hugging Face/Colab Permission Errors) ---
os.environ['HOME'] = '/tmp'
os.environ['MPLCONFIGDIR'] = '/tmp/matplotlib_cache'
os.environ['STREAMLIT_SERVER_UNSAFE_WRITE_ENABLED'] = 'true'


# --- 1. CONFIGURATION AND UTILITY FUNCTIONS ---

APP_NAME = "VJH HealthWatch: Sentinel Health"
st.set_page_config(layout="wide", page_title=APP_NAME)

BASE_LAT = 28.7041
BASE_LON = 77.1025
TARGET_PERIOD_DAYS = 100

DISEASE_OPTIONS = ['Dengue', 'Leptospirosis', 'Malaria', 'Chikungunya', 'Typhoid']
STATE_OPTIONS = [
    'Nationwide', # Default Option
    'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh', 'Goa',
    'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka', 'Kerala',
    'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram', 'Nagaland',
    'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura',
    'Uttar Pradesh', 'Uttarakhand', 'West Bengal',
    'Andaman and Nicobar Islands', 'Chandigarh', 'Dadra and Nagar Haveli and Daman and Diu',
    'Delhi', 'Lakshadweep', 'Puducherry', 'Jammu and Kashmir', 'Ladakh'
]
POPULATION_MOCK = 10000000

STATE_CITY_MOCKUP = {
    'Maharashtra': {'Mumbai': [19.0760, 72.8777], 'Pune': [18.5204, 73.8567], 'Nagpur': [21.1458, 79.0882], 'Nashik': [19.9975, 73.7898]},
    'Delhi': {'New Delhi': [28.6139, 77.2090], 'Gurgaon (HR)': [28.4595, 77.0266], 'Noida (UP)': [28.5355, 77.3910], 'Faridabad': [28.4089, 77.3178]},
    'Tamil Nadu': {'Chennai': [13.0827, 80.2707], 'Coimbatore': [11.0168, 76.9558], 'Madurai': [9.9252, 78.1198], 'Tiruchirappalli': [10.7905, 78.7047]},
    'Telangana': {'Hyderabad': [17.3850, 78.4867], 'Warangal': [17.9689, 79.5941], 'Karimnagar': [18.4340, 79.1288], 'Nizamabad': [18.6631, 78.0934]},
    'Karnataka': {'Bengaluru': [12.9716, 77.5946], 'Mysuru': [12.2958, 76.6394], 'Mangaluru': [12.9141, 74.8560], 'Hubli': [15.3647, 75.1240]},
    'West Bengal': {'Kolkata': [22.5726, 88.3639], 'Howrah': [22.5831, 88.3188], 'Siliguri': [26.7214, 88.3976], 'Durgapur': [23.4900, 87.3100]},
    'Kerala': {'Kochi': [10.8505, 76.2711]},
    'Chhattisgarh': {'Raipur': [21.2514, 81.6296]},
    'Gujarat': {'Ahmedabad': [23.0225, 72.5714]},
    'Rajasthan': {'Jaipur': [26.9124, 75.7873]},
    'Uttar Pradesh': {'Lucknow': [26.8467, 80.9462]},
    'Andhra Pradesh': {'Visakhapatnam': [17.6868, 83.2185]},
    'Assam': {'Guwahati': [26.1445, 91.7362]},
    'Arunachal Pradesh': {'Itanagar': [27.0610, 93.6050]},
}

# --- Mockup Dictionary for Map Centering (approximate centers and appropriate zoom levels) ---
STATE_ZOOM_MOCKUP = {
    'Andhra Pradesh': {'center': [15.9179, 80.0000], 'zoom': 6},
    'Arunachal Pradesh': {'center': [28.0, 94.5], 'zoom': 6},
    'Assam': {'center': [26.2006, 92.9376], 'zoom': 6},
    'Bihar': {'center': [25.0961, 85.3131], 'zoom': 6},
    'Chhattisgarh': {'center': [21.5, 82.0], 'zoom': 6},
    'Goa': {'center': [15.2993, 74.1240], 'zoom': 8},
    'Gujarat': {'center': [22.2587, 71.1924], 'zoom': 6},
    'Haryana': {'center': [29.0588, 76.0856], 'zoom': 7},
    'Himachal Pradesh': {'center': [31.1048, 77.1734], 'zoom': 7},
    'Jharkhand': {'center': [23.6102, 85.2799], 'zoom': 7},
    'Karnataka': {'center': [14.0, 76.0], 'zoom': 6},
    'Kerala': {'center': [10.0, 76.5], 'zoom': 7},
    'Madhya Pradesh': {'center': [22.9941, 78.6015], 'zoom': 6},
    'Maharashtra': {'center': [19.5, 75.0], 'zoom': 6},
    'Manipur': {'center': [24.6637, 93.9063], 'zoom': 7},
    'Meghalaya': {'center': [25.4670, 91.3662], 'zoom': 7},
    'Mizoram': {'center': [23.1645, 92.9376], 'zoom': 7},
    'Nagaland': {'center': [26.1584, 94.5624], 'zoom': 7},
    'Odisha': {'center': [20.9517, 85.0985], 'zoom': 6},
    'Punjab': {'center': [31.1471, 75.3412], 'zoom': 7},
    'Rajasthan': {'center': [27.0238, 74.2179], 'zoom': 5},
    'Sikkim': {'center': [27.5330, 88.5122], 'zoom': 8},
    'Tamil Nadu': {'center': [11.0, 78.0], 'zoom': 6},
    'Telangana': {'center': [17.0, 79.0], 'zoom': 6},
    'Tripura': {'center': [23.9408, 91.9882], 'zoom': 7},
    'Uttar Pradesh': {'center': [26.8467, 80.9462], 'zoom': 6},
    'Uttarakhand': {'center': [30.0668, 79.0193], 'zoom': 7},
    'West Bengal': {'center': [23.0, 88.0], 'zoom': 6},
    'Andaman and Nicobar Islands': {'center': [10.0, 92.5], 'zoom': 5},
    'Chandigarh': {'center': [30.7333, 76.7794], 'zoom': 10},
    'Dadra and Nagar Haveli and Daman and Diu': {'center': [20.4182, 73.0166], 'zoom': 7},
    'Delhi': {'center': [28.6, 77.2], 'zoom': 9},
    'Lakshadweep': {'center': [10.5, 72.8], 'zoom': 7},
    'Puducherry': {'center': [11.9161, 79.8083], 'zoom': 8},
    'Jammu and Kashmir': {'center': [34.0, 74.5], 'zoom': 6},
    'Ladakh': {'center': [34.5, 77.5], 'zoom': 6}
}

REAL_WORLD_ROUTE = [
    [19.0760, 72.8777], [22.57, 75.87], [25.0, 77.0], [28.7041, 77.1025]
]

# Load GeoJSON (State Boundaries) once for Choropleth Map (Optional/Disabled for stability)
@st.cache_resource
def load_geojson_data():
    GEOJSON_PATH = 'india_states.geojson'

    if not os.path.exists(GEOJSON_PATH):
        try:
            GEOJSON_URL = 'https://raw.githubusercontent.com/datameet/maps/master/States/gadm36_IND_1.json'
            response = requests.get(GEOJSON_URL, timeout=15)
            response.raise_for_status()
            with open(GEOJSON_PATH, 'w', encoding='utf-8') as f:
                f.write(response.text)
        except Exception:
            return None
    try:
        with open(GEOJSON_PATH, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception:
        return None

INDIA_GEOJSON = load_geojson_data()

# --- UI STYLE INJECTION ---
STYLING = """
<style>
.stApp { background-color: #1a1a2e; color: #f0f0f0; }
[data-testid="stSidebar"] { background-color: #161625; color: #f0f0f0; padding-top: 50px; }
.stCard { background-color: #24243e; border: 1px solid #3d3d5c; border-radius: 10px; padding: 10px; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2); }
h1 { color: #8c8cff; }
h3 { color: #b3b3e6; }
[data-testid="stMetricValue"] { color: #f0f0f0 !important; }
.stAlert, [data-testid="stExpander"] { background-color: #1f1f3a !important; }
</style>
"""

# --- THEORETICAL VECTOR INDEX CALCULATION ---
def calculate_theoretical_vector_index(weather_df):
    start_lag = 14
    end_lag = 28
    lagged_data = weather_df.iloc[-(end_lag + 1):-start_lag].copy()
    if lagged_data.empty: return 0

    rainfall_score = np.clip(lagged_data['Rainfall_mm'].sum() / 50.0, 0, 10)
    optimal_temp = 27.5
    temp_deviation = np.abs(lagged_data['Avg_Temp_C'].mean() - optimal_temp)
    temp_score = 10 - np.clip(temp_deviation * 2, 0, 10)

    tvi = round((rainfall_score * 0.4) + (temp_score * 0.6), 1)
    breteau_index = np.clip(tvi * 10, 0, 100)
    return breteau_index


# --- ACTION TEXT GENERATION (NO LANGUAGE SUPPORT) ---
def get_action_text(risk_level, disease):
    if risk_level == 'RED':
        return f"🚨 **IMMEDIATE ACTION REQUIRED:** DANGER! Expect a major surge in {disease} cases. **Act NOW:** Implement immediate control measures and screen high-risk areas."
    elif risk_level == 'YELLOW':
        return f"⚠️ **CAUTIONARY MEASURES:** CAUTION! Stable but elevated risk of {disease}. Be careful: Increase surveillance and educate the public on prevention."
    else: # GREEN
        return f"✅ **MAINTENANCE MODE:** SAFE ZONE! Risk of {disease} is low. No immediate action, but maintain routine sanitation checks."

def create_time_series_features(df, lags=[1, 7, 14]):
    for lag in lags:
        df[f'Cases_Lag_{lag}'] = df['Daily_Cases'].shift(lag)
        df[f'Rainfall_Lag_{lag}'] = df['Rainfall_mm'].shift(lag)
        df[f'Temp_Lag_{lag}'] = df['Avg_Temp_C'].shift(lag)
    return df.dropna()

def get_color(risk):
    if risk >= 6.0: return 'darkred'
    elif risk >= 3.0: return 'orange'
    else: return 'green'

def fetch_real_case_data(state, disease):
    if disease == 'Dengue' and state == 'Delhi':
        days = pd.date_range(end=datetime.now(), periods=100, freq='D')
        cases = (np.sin(np.linspace(0, 3*np.pi, 100)) * 100 + 200).astype(int)
        cases[cases < 0] = 50
        real_data = pd.DataFrame({'Date': days, 'Cases': cases})
        return real_data
    return False

# --- 2. DATA LOADING (Mock Data Integration) ---
@st.cache_data
def load_and_process_data():

    end_date = datetime.now()

    # --- Stable Mock Data ---
    np.random.seed(42)
    days = pd.date_range(end=end_date, periods=TARGET_PERIOD_DAYS, freq='D')
    rainfall_data = np.random.normal(loc=15, scale=5, size=TARGET_PERIOD_DAYS).clip(min=0).round(1)
    temp_data = np.random.normal(loc=30, scale=3, size=TARGET_PERIOD_DAYS).clip(min=0).round(1)
    weather_df = pd.DataFrame({'Date': days, 'Avg_Temp_C': temp_data, 'Rainfall_mm': rainfall_data})
    weather_df.set_index('Date', inplace=True)


    # 2b. COMPREHENSIVE CASE DATASET
    data_list = []
    # Note: STATE_OPTIONS now includes 'Nationwide', but we only use the actual states for data simulation
    actual_states = [s for s in STATE_OPTIONS if s != 'Nationwide']
    state_factors_base = {state: 1.0 + (i % 5) * 0.2 for i, state in enumerate(actual_states)}
    disease_factors = {'Dengue': 1.0, 'Leptospirosis': 0.5, 'Malaria': 1.3, 'Chikungunya': 0.9, 'Typhoid': 0.7}

    for state in actual_states:
        s_factor = state_factors_base.get(state, 1.0)
        for disease, d_factor in disease_factors.items():

            days = weather_df.index
            np.random.seed(hash(state + disease) % 100)

            base_case_level = 100 if disease in ['Dengue', 'Malaria'] else 50
            base_cases = np.sin(np.linspace(0, 2*np.pi, len(days))) * 50 + base_case_level
            daily_cases = (base_cases * s_factor * d_factor + np.random.normal(0, 15, len(days))).astype(int)
            daily_cases[daily_cases < 0] = 0

            ts_df_state_disease = weather_df.copy()
            ts_df_state_disease['Daily_Cases'] = daily_cases
            ts_df_state_disease['State'] = state
            ts_df_state_disease['Disease'] = disease
            ts_df_state_disease['Data_Source'] = 'SIMULATED'

            data_list.append(ts_df_state_disease.reset_index())

    full_timeseries_df = pd.concat(data_list, ignore_index=True)

    return full_timeseries_df

full_timeseries_df = load_and_process_data()


# --- 4. SPREAD TRACKING MODULE ---
def create_spread_tracking_simulation():
    st.subheader("Disease Spread Simulation")

    spread_map = folium.Map(location=[20.5937, 78.9629], zoom_start=4.5, tiles='cartodbdarkmatter')

    AntPath(locations=REAL_WORLD_ROUTE, color="#FF0000", weight=3, delay=1000, dash_array=[10, 20]).add_to(spread_map)

    for city, location in {'Mumbai (Origin)': REAL_WORLD_ROUTE[0], 'Delhi (Hub)': REAL_WORLD_ROUTE[-1]}.items():
        folium.Marker(location, popup=city, icon=folium.Icon(color='red' if 'Origin' in city else 'blue', icon='plane', prefix='fa')).add_to(spread_map)

    folium_static(spread_map, width=380, height=200)

# --- 5. REAL-TIME ENVIRONMENTAL DATA MODULE ---
def create_environmental_data(ts_df, data_state):
    st.subheader(f"Real-time Environmental Data ({data_state})")

    current_data = ts_df.tail(1)
    temp = current_data['Avg_Temp_C'].iloc[0] if not current_data.empty else 0
    rain = current_data['Rainfall_mm'].iloc[0] if not current_data.empty else 0
    humidity = random.randint(60, 85)

    col1, col2, col3 = st.columns(3)
    col1.metric("Temperature", f"{temp:.1f} °C", "Normal")
    col2.metric("Rainfall", f"{rain:.1f} mm", "Normal")
    col3.metric("Humidity", f"{humidity}%", "Normal")

# --- 6. SOCIAL SENTIMENT MODULE (MOCK) ---
def create_social_sentiment():
    st.subheader("Social Sentiment Analysis")

    data = {'Fever Reports': 450, 'Water Logging': 790, 'Sanitation Comp.': 260}
    df = pd.DataFrame(list(data.items()), columns=['Category', 'Count'])

    # FIX: Increase figure size slightly while keeping font sizes small and using tight_layout
    fig, ax = plt.subplots(figsize=(3.5, 2.0))
    colors = ['#3d3d5c', '#5c5c8a', '#8c8cff']
    ax.bar(df['Category'], df['Count'], color=colors)

    ax.set_facecolor('#24243e')
    fig.patch.set_facecolor('#24243e')

    # Smaller font sizes for readability in a confined space
    ax.tick_params(axis='x', rotation=45, colors='#f0f0f0', labelsize=7) # Slightly larger font
    ax.tick_params(axis='y', colors='#f0f0f0', labelsize=7) # Slightly larger font
    ax.set_ylabel("Report Count", color='#f0f0f0', fontsize=7) # Slightly larger font
    ax.title.set_color('#b3b3e6')
    ax.tick_params(axis='both', which='major', pad=0)

    # Hide top/right spines and set bottom/left colors
    ax.spines['bottom'].set_color('#f0f0f0')
    ax.spines['left'].set_color('#f0f0f0')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout() # Adjust layout to prevent clipping

    # Use container width=True is essential for fitting the column size
    st.pyplot(fig, use_container_width=True)
    st.caption("Early warning signal detected in Water Logging chatter.")


# --- 7. FORECASTING MODULE (Integrated into UI) ---
def create_forecast_chart(full_timeseries_df, selected_state, selected_disease):

    # For the forecast chart, if 'Nationwide' is selected, we must pick a state for ML data.
    data_state = selected_state if selected_state != 'Nationwide' else 'Maharashtra'

    ts_df = full_timeseries_df[
        (full_timeseries_df['State'] == data_state) &
        (full_timeseries_df['Disease'] == selected_disease)
    ].set_index('Date')

    if ts_df.empty:
        st.error(f"Error: No data found for {selected_disease} in {data_state}.")
        return 0, 'green', pd.DataFrame(), 0, "GREEN" # Return defaults on error

    # Run ML Model (Same Robust Logic)
    FEATURES = ['Rainfall_mm', 'Avg_Temp_C', 'Cases_Lag_1', 'Rainfall_Lag_1', 'Temp_Lag_1', 'Cases_Lag_7', 'Rainfall_Lag_7', 'Temp_Lag_7', 'Cases_Lag_14', 'Rainfall_Lag_14', 'Temp_Lag_14']
    model_df = create_time_series_features(ts_df.copy())
    X, y = model_df[FEATURES], model_df['Daily_Cases']
    xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, random_state=42).fit(X, y)

    forecast_period = 14
    predicted_cases = []
    current_data = ts_df.tail(14).copy()
    forecast_dates = pd.date_range(start=ts_df.index[-1] + pd.Timedelta(days=1), periods=forecast_period, freq='D')
    mean_rainfall, mean_temp = ts_df['Rainfall_mm'].mean(), ts_df['Avg_Temp_C'].mean()

    for i in range(forecast_period):
        X_today = pd.DataFrame({'Rainfall_mm': [mean_rainfall], 'Avg_Temp_C': [mean_temp], 'Cases_Lag_1': [current_data['Daily_Cases'].iloc[-1]], 'Rainfall_Lag_1': [current_data['Rainfall_mm'].iloc[-1]], 'Temp_Lag_1': [current_data['Avg_Temp_C'].iloc[-1]], 'Cases_Lag_7': [current_data['Daily_Cases'].iloc[-7]], 'Rainfall_Lag_7': [current_data['Rainfall_mm'].iloc[-7]], 'Temp_Lag_7': [current_data['Avg_Temp_C'].iloc[-7]], 'Cases_Lag_14': [current_data['Daily_Cases'].iloc[0]], 'Rainfall_Lag_14': [current_data['Rainfall_mm'].iloc[0]], 'Temp_Lag_14': [current_data['Avg_Temp_C'].iloc[0]],})
        prediction = int(max(0, round(xgb_model.predict(X_today)[0])))
        predicted_cases.append(prediction)
        new_row = pd.Series({'Daily_Cases': prediction, 'Rainfall_mm': mean_rainfall, 'Avg_Temp_C': mean_temp}, name=forecast_dates[i])
        current_data = pd.concat([current_data.iloc[1:], new_row.to_frame().T], axis=0)

    # Output Metrics
    forecast_output = pd.DataFrame({'Predicted_Cases': predicted_cases}, index=forecast_dates)
    historical_avg = ts_df['Daily_Cases'].tail(14).mean()
    forecast_avg = forecast_output['Predicted_Cases'].mean()
    change_percent = ((forecast_avg / historical_avg) - 1) * 100

    # Determine Traffic Light Status
    if change_percent >= 25: plot_color, increase_label, risk_level = 'darkred', f"↑ {change_percent:.1f}% Increase", "RED"
    elif change_percent >= 5: plot_color, increase_label, risk_level = 'orange', f"↑ {change_percent:.1f}% Potential Increase", "YELLOW"
    else: plot_color, increase_label, risk_level = 'green', f"↓ {change_percent:.1f}% Decrease", "GREEN"

    # Display Top Metrics
    col_cases, col_chart = st.columns([1.5, 2])
    col_cases.metric("Predicted Cases (14 Days)", f"{forecast_avg*14:.0f} cases", increase_label, delta_color="inverse")

    # Small Chart for the card
    plot_data = pd.concat([ts_df.tail(30), forecast_output], axis=0)
    fig, ax = plt.subplots(figsize=(2, 2))
    ax.plot(plot_data.index[-14:], forecast_output['Predicted_Cases'], color=plot_color, linestyle='--', linewidth=1)
    ax.fill_between(plot_data.index[-14:], forecast_output['Predicted_Cases'], color=plot_color, alpha=0.3)
    ax.set_facecolor('#24243e')
    fig.patch.set_facecolor('#24243e')
    ax.axis('off')
    col_chart.pyplot(fig)

    # Return data for Actionable Alert Box
    return change_percent, plot_color, forecast_output, historical_avg, risk_level

# --- PAGE FUNCTIONS ---

def news_overview_page(full_timeseries_df, selected_state, selected_disease):

    forecast_header = selected_state if selected_state != 'Nationwide' else 'National'
    data_state_metrics = selected_state if selected_state != 'Nationwide' else 'Maharashtra'

    st.title(f"{APP_NAME} - Nationwide Health Overview 📰")
    st.markdown("---")

    # 1. Prediction/Risk Section
    st.subheader(f"Forecast and Risk for {forecast_header}")

    # Use two columns for the prediction block and the alert box
    col_pred_chart, col_alert_box = st.columns([2, 1])

    with col_pred_chart:
        # This function prints the Prediction Metric and small Chart
        change_percent, plot_color, forecast_output, historical_avg, risk_level = create_forecast_chart(
            full_timeseries_df, selected_state, selected_disease
        )

    with col_alert_box:
        # Display the Risk Status Banner
        st.markdown(f"### <span style='color:{plot_color};'>**{risk_level} ALERT**</span>", unsafe_allow_html=True)
        st.caption(f"Current Risk Status based on 14-Day Forecast for {forecast_header}.")
        st.markdown("---")

        # Display the Actionable Insight Box (Cautionary Statement)
        action_text = get_action_text(risk_level, selected_disease)

        if risk_level == 'RED':
            st.error(action_text)
        elif risk_level == 'YELLOW':
            st.warning(action_text)
        else:
            st.success(action_text)

    st.markdown("---") # Separator after the main risk section

    # 2. News and Quick Metrics Section
    col_news, col_quick_metrics = st.columns([2, 1])

    with col_news:
        st.subheader("Local Impact News Feed (MOCK)")
        date_today = datetime.now().strftime('%b %d')
        date_yesterday = (datetime.now() - timedelta(days=1)).strftime('%b %d')
        date_two_days = (datetime.now() - timedelta(days=2)).strftime('%b %d')

        st.markdown(f"**{date_today}**: High-risk advisory issued for Pune, Maharashtra (linked to high TBI).")
        st.markdown(f"**{date_yesterday}**: Sanitation drive launched in Delhi after Social Sentiment spikes in 'Water Logging' reports.")
        st.markdown(f"**{date_two_days}**: {data_state_metrics} reports low {selected_disease} risk; Green status maintained.")


    with col_quick_metrics:
        st.subheader("Quick Metrics")

        proxy_ts_df = full_timeseries_df[full_timeseries_df['State'] == data_state_metrics].tail(1)
        temp = proxy_ts_df['Avg_Temp_C'].iloc[0] if not proxy_ts_df.empty else 0
        rain = proxy_ts_df['Rainfall_mm'].iloc[0] if not proxy_ts_df.empty else 0

        st.metric(f"Avg. Temp ({data_state_metrics})", f"{temp:.1f} °C", "Normal")
        st.metric(f"Avg. Rainfall ({data_state_metrics})", f"{rain:.1f} mm", "Normal")

        trend_df = full_timeseries_df[full_timeseries_df['Disease'] == selected_disease]
        if not trend_df.empty:
            historical_avg = trend_df['Daily_Cases'].mean()
            current_cases = trend_df['Daily_Cases'].tail(36).mean()
            trend_change = ((current_cases / historical_avg) - 1) * 100

            st.metric("National 7-Day Trend", f"{current_cases:.0f} cases/day", f"{trend_change:.1f}%", delta_color='inverse' if trend_change > 0 else 'normal')


def map_analysis_page(full_timeseries_df, selected_state, selected_disease):
    st.title("Geospatial Risk Analysis (Nationwide Spread) 🌍")
    st.markdown("---")

    col_map, col_tracking = st.columns([3, 1])

    with col_map:
        st.subheader(f"{selected_state} Hotspot Map for {selected_disease}")

        # --- Dynamic Map Centering Logic ---
        default_center = [22.0, 78.0]
        default_zoom = 4.5

        # If Nationwide is selected, use default zoom/center (All India)
        if selected_state == 'Nationwide' or selected_state not in STATE_ZOOM_MOCKUP:
            map_center = default_center
            map_zoom = default_zoom
            map_is_zoomed = False
        else:
            # If a specific state is selected and exists in the mockup, zoom in
            map_center = STATE_ZOOM_MOCKUP[selected_state]['center']
            map_zoom = STATE_ZOOM_MOCKUP[selected_state]['zoom']
            map_is_zoomed = True

        # Map generation: Use dynamically calculated center and zoom
        hotspot_map = folium.Map(location=map_center, zoom_start=map_zoom, tiles='cartodbdarkmatter')

        # --- Data Preparation for Map ---
        latest_date = full_timeseries_df['Date'].max()

        # Get all cities in the mockup
        all_cities = [
            {'City': city, 'Latitude': coords[0], 'Longitude': coords[1], 'State': state, 'Risk_Index': (random.random() * 8) + 2}
            for state, cities in STATE_CITY_MOCKUP.items()
            for city, coords in cities.items()
        ]
        major_cities_df = pd.DataFrame(all_cities)

        # FIX: Initialize map_dots_df to an empty DataFrame to prevent UnboundLocalError
        map_dots_df = pd.DataFrame()

        # Determine which cities to plot:
        if map_is_zoomed and selected_state in STATE_CITY_MOCKUP:
            # If zoomed to a specific state, show all available cities in that state
            map_dots_df = major_cities_df[major_cities_df['State'] == selected_state].copy()

            # Use mock risk data based on disease for visual distinction (ensure data exists)
            if not map_dots_df.empty:
                 # Adjust risk factor to reflect disease selection (higher index = generally higher mock risk)
                disease_factor = (DISEASE_OPTIONS.index(selected_disease) + 1) / 3
                map_dots_df.loc[:, 'Risk_Index'] = (map_dots_df['Risk_Index'] * disease_factor).clip(1, 9)

        # Fallback to National view if 'Nationwide' selected OR if zoomed state has no cities mocked (map_dots_df.empty)
        if selected_state == 'Nationwide' or map_dots_df.empty:
            # Default National view logic (show cities in top high-risk states globally)
            state_risk_df = full_timeseries_df[
                (full_timeseries_df['Disease'] == selected_disease) &
                (full_timeseries_df['Date'] == latest_date)
            ].groupby('State')['Daily_Cases'].mean().reset_index()

            max_cases = state_risk_df['Daily_Cases'].max()
            state_risk_df['Risk_Score'] = (state_risk_df['Daily_Cases'] / max_cases) * 10 if max_cases > 0 else 0

            # Identify top half riskiest states nationally
            top_half_states = state_risk_df.nlargest(len(state_risk_df) // 2, 'Daily_Cases')['State'].tolist()

            map_dots_df = major_cities_df[major_cities_df['State'].isin(top_half_states)]


        # Display all city dots (for visual saturation)
        for index, row in map_dots_df.iterrows():
            popup_text = f"<b>{row['City']}</b><br>Risk: {row['Risk_Index']:.1f}"
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                radius=8,
                color=get_color(row['Risk_Index']),
                fill=True,
                fill_color=get_color(row['Risk_Index']),
                fill_opacity=0.8,
                popup=popup_text
            ).add_to(hotspot_map)

        folium_static(hotspot_map, width=950, height=550)

    # --- Spread Tracking Column (Right) ---
    with col_tracking:
        st.subheader("Spread Simulation")

        # Display Spread Tracking & Sentiment (Right Panel)
        create_spread_tracking_simulation()
        st.markdown("---")

        st.subheader("Top Risk Summary")

        # Recalculate the top states for display
        state_risk_df = full_timeseries_df[
            (full_timeseries_df['Disease'] == selected_disease) &
            (full_timeseries_df['Date'] == latest_date)
        ].groupby('State')['Daily_Cases'].mean().reset_index()
        top_states_display = state_risk_df.nlargest(4, 'Daily_Cases')['State'].tolist()

        st.info(f"Top 4 Riskiest States: {', '.join(top_states_display)}")
        st.markdown("---")

        # SOCIAL SENTIMENT CARD (MOVED FROM OVERVIEW TO MAP ANALYSIS RIGHT PANEL)
        with st.container(border=True):
            create_social_sentiment()


def alerts_page():
    st.title("Historical Alerts and Incidents 🔔")
    st.markdown("---")
    st.warning("This page simulates the historical records generated by the VJH HealthWatch system.")
    st.info("The Dashboard buttons are fully functional, allowing navigation back to the main view.")

    data = {'Date': pd.to_datetime(['2025-09-01', '2025-09-15', '2025-10-01', '2025-10-10']),
            'State': ['Maharashtra', 'NCT of Delhi', 'Telangana', 'Maharashtra'],
            'Disease': ['Dengue', 'Malaria', 'Dengue', 'Leptospirosis'],
            'Risk Level': ['RED (7.5)', 'YELLOW (4.2)', 'RED (6.1)', 'GREEN (1.1)'],
            'Action': ['Spray/Draining order issued.', 'Increased public education.', 'Full health lockdown.', 'Routine check maintained.']}

    st.dataframe(pd.DataFrame(data), use_container_width=True)

def settings_page():
    st.title("System Settings and Data Configuration ⚙️")
    st.markdown("---")
    st.info("This section displays the configuration parameters and data status.")

    col1, col2 = st.columns(2)

    with col1:
        st.subheader("ML Model Parameters")
        st.metric("Model Type", "XGBoost Regressor")
        st.metric("Prediction Window", "14 Days")
        st.metric("Epidemiological Lag", "14-28 Days (Vector Lifecycle)")

    with col2:
        st.subheader("Data Source Status")
        st.metric("Weather API", "Active (Mock Data)")
        st.metric("Case Data", "SIMULATED (High Stability)")
        st.metric("TBI Threshold", "50 (Critical Breteau Index)")

    st.markdown("---")
    st.caption("All model parameters and data links are configured for rapid deployment.")

# --- MAIN ROUTER ---

def main():

    # Initialize session state for navigation
    if 'page' not in st.session_state:
        st.session_state.page = 'overview'

    st.markdown(STYLING, unsafe_allow_html=True)

    # --- SIDEBAR AND FILTERS ---
    with st.sidebar:
        # Use lambda functions to set session state when a button is clicked (Navigation Fix)
        st.markdown("## **VJH**", unsafe_allow_html=True)
        # The primary button should reflect the current page
        st.button("Overview", use_container_width=True, type="primary" if st.session_state.page == 'overview' else 'secondary', on_click=lambda: st.session_state.update(page='overview'))
        st.button("Map Analysis", use_container_width=True, type="primary" if st.session_state.page == 'map' else 'secondary', on_click=lambda: st.session_state.update(page='map'))
        st.button("Alerts", use_container_width=True, type="primary" if st.session_state.page == 'alerts' else 'secondary', on_click=lambda: st.session_state.update(page='alerts'))
        st.button("Settings", use_container_width=True, type="primary" if st.session_state.page == 'settings' else 'secondary', on_click=lambda: st.session_state.update(page='settings'))
        st.markdown("---")

        st.subheader("Location & Disease Filters")

        # Set default state to "Nationwide" (index 0)
        selected_state = st.selectbox("State", STATE_OPTIONS, index=0)

        # Set default disease to Malaria (index 2 in DISEASE_OPTIONS)
        selected_disease = st.selectbox("Disease", DISEASE_OPTIONS, index=2)

    # --- PAGE ROUTING LOGIC ---
    if st.session_state.page == 'overview':
        news_overview_page(full_timeseries_df, selected_state, selected_disease)
    elif st.session_state.page == 'map':
        map_analysis_page(full_timeseries_df, selected_state, selected_disease)
    elif st.session_state.page == 'alerts':
        alerts_page()
    elif st.session_state.page == 'settings':
        settings_page()

if __name__ == "__main__":
    main()

Overwriting app.py
